<a href="https://colab.research.google.com/github/iamcoffeeboy0/lb-scrapy/blob/main/LETTERBOXD_STATS_2023_v4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CREDITS**

https://github.com/dericored/data-exploration-letterboxd by **dericored**

https://github.com/iamcoffeeboy0/lb-scrapy by **iamcoffeeboy0**






In [ ]:
#@title STEP 0 - Requeriments
%%capture
!pip install scrapy
!git clone https://github.com/iamcoffeeboy0/lb-scrapy.git
!pip install pandas
!pip isntall seaborn
!pip install matplotlib
!pip install plotly
!apt-get install -y fonts-nanum
!pip install kaleido
!rm -rf /root/.cache/matplotlib/*

import seaborn as sns
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import plotly.graph_objects as go
import csv
from google.colab import files
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from matplotlib.patches import FancyBboxPatch
from matplotlib.path import get_path_collection_extents
from itertools import product
import numpy as np
import six
import zipfile
import os
from google.colab import files
from itertools import product
import numpy as np
import six
# Ruta de la carpeta a crear
carpeta_destino = '/content/img'

# Verificar si la carpeta no existe antes de crearla
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)
    print(f'Se ha creado la carpeta {carpeta_destino}')
else:
    print(f'La carpeta {carpeta_destino} ya existe')

In [ ]:
#@title STEP 1 - Extract data, enter your letterboxd username and wait.
!scrapy runspider /content/lb-scrapy/diary_spider.py -o resultado.json

In [ ]:
#@title Films Watched
# open file and store to df
f = open('/content/resultado.json')
df = pd.read_json(f)
df = pd.to_datetime(df['watch_date'])

# find total movies
total = len(df)

# find period (in months) between first and last record
months = (max(df).year - min(df).year)*12 + (max(df).month - min(df).month)

per_month = total/months

# find period (in weeks) between first and last record
weeks = (max(df) - min(df)).days//7

per_week = total/weeks
# configure font
plt.rcParams['font.family'] = []
plt.rcParams['font.serif'] = []
plt.style.use('dark_background')
fig = plt.figure(figsize=(6, 2))  # Adjust the values as needed

# plot stats as texts
plt.text(x=0.2, y=0.3, s='Films watched', va='center', ha='center', fontsize=10, color='gray')
plt.text(x=0.2, y=0.5, s=total, va='center', ha='center', fontsize=20, fontweight='semibold')
plt.text(x=0.5, y=0.3, s='Average per month', va='center', ha='center', fontsize=10, color='gray')
plt.text(x=0.5, y=0.5, s='{:.1f}'.format(per_month), va='center', ha='center', fontsize=20, fontweight='semibold')
plt.text(x=0.8, y=0.3, s='Average per week', va='center', ha='center', fontsize=10, color='gray')
plt.text(x=0.8, y=0.5, s='{:.1f}'.format(per_week), va='center', ha='center', fontsize=20, fontweight='semibold')
plt.text(x=0.5, y=0.8, s='from {}  ~  {}'.format(min(df).strftime('%B %d, %Y'), max(df).strftime('%B %d, %Y')), va='center', ha='center', fontsize=10, color='gray')
plt.axis('off')
plt.savefig('/content/img/0_Stats_General.png', dpi=300, bbox_inches='tight')

In [ ]:
#@title Number of unique directors, actors and countries
df = pd.read_json('/content/resultado.json')

# store column data to respective variables
director = df['director']
actor = df['actor']
country = df['country']
runtime = df['runtime']
released_year = df['released_year']

# explode the list and get the number of unique directors, actors and countries
directors = director.explode().nunique()
actors = actor.explode().nunique()
countries = country.explode().nunique()

# get total runtime
total_runtime = sum(runtime)

# get the earliest and latest released year
earliest = min(released_year)
latest = max(released_year)

# configure font
plt.rcParams['font.family'] = []
plt.rcParams['font.serif'] = []
plt.style.use('dark_background')
fig = plt.figure(figsize=(3, 5))  # Adjust the values as needed

# plot stats as texts
plt.text(x=1.25, y=0.85, s='Stats', va='center', ha='right', fontsize=15, color='gray')
plt.text(x=0.1, y=0.75, s='Directors', va='center', ha='left', fontsize=10, color='gray')
plt.text(x=1.25, y=0.75, s=f'{directors:,}', va='center', ha='right', fontsize=20, fontweight='semibold')
plt.text(x=0.1, y=0.6, s='Actors', va='center', ha='left', fontsize=10, color='gray')
plt.text(x=1.25, y=0.6, s=f'{actors:,}', va='center', ha='right', fontsize=20, fontweight='semibold')
plt.text(x=0.1, y=0.45, s='Countries', va='center', ha='left', fontsize=10, color='gray')
plt.text(x=1.25, y=0.45, s=f'{countries:,}', va='center', ha='right', fontsize=20, fontweight='semibold')
plt.text(x=0.1, y=0.3, s='Runtime (mins)', va='center', ha='left', fontsize=10, color='gray')
plt.text(x=1.25, y=0.3, s=f'{total_runtime:,}', va='center', ha='right', fontsize=20, fontweight='semibold')
plt.text(x=0.1, y=0.1, s=f'from films released in {earliest} - {latest}', va='center', ha='left', fontsize=10, color='gray', fontweight='semibold')
plt.axis('off')
plt.savefig("/content/img/1_Stats_Additional.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Directors

df = pd.read_json('/content/resultado.json')

top = df['director'].explode().value_counts()
unique_directors = top.index.tolist()

directores_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for director in unique_directors:
    search_results = df[df['director'].apply(lambda x: director in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    directores_lista.append(director)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_directores = pd.DataFrame({
    'Director': directores_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})

df_directores = df_directores[df_directores["Movies"] > 1]
df_directores = df_directores.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_directores[:(12 if len(df_directores)>12 else len(df_directores))], x="Movies", y="Director")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Directors '+r'$(n \geq 2)$',  fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/2_Most_Watched_Directors.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Directors

df = pd.read_json('/content/resultado.json')

top = df['director'].explode().value_counts()
unique_directors = top.index.tolist()

directores_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for director in unique_directors:
    search_results = df[df['director'].apply(lambda x: director in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    directores_lista.append(director)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_directores = pd.DataFrame({
    'Director': directores_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})

df_directores = df_directores[df_directores["Movies"] > 1]
df_directores = df_directores.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_directores[:(12 if len(df_directores)>12 else len(df_directores))], x="Personal Rating", y="Director")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Directors '+r'$(n \geq 2)$',  fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/3_Favorite_Directors.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Actors
df = pd.read_json('/content/resultado.json')

top = df['actor'].explode().value_counts()
unique_actors = top[top > 1].index.tolist()

actores_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for actor in unique_actors:
    search_results = df[df['actor'].apply(lambda x: actor in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    actores_lista.append(actor)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_actores = pd.DataFrame({
    'Actor': actores_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_actores = df_actores[df_actores["Movies"] > 1]
df_actores = df_actores.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_actores[:(12 if len(df_actores)>12 else len(df_actores))], x="Movies", y="Actor")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Actors '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/4_Most_Watched_Actors.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Actors
df = pd.read_json('/content/resultado.json')

top = df['actor'].explode().value_counts()
unique_actors = top[top > 1].index.tolist()

actores_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for actor in unique_actors:
    search_results = df[df['actor'].apply(lambda x: actor in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    actores_lista.append(actor)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_actores = pd.DataFrame({
    'Actor': actores_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_actores = df_actores[df_actores["Movies"] > 1]
df_actores = df_actores.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_actores[:(12 if len(df_actores)>12 else len(df_actores))], x="Personal Rating", y="Actor")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Actors '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/5_Favorite_Actors.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Cinematographers
df = pd.read_json('/content/resultado.json')

top = df['cinematographer'].explode().value_counts()
unique_cinematographers = top[top > 1].index.tolist()

cinematographeres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for cinematographer in unique_cinematographers:
    search_results = df[df['cinematographer'].apply(lambda x: cinematographer in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    cinematographeres_lista.append(cinematographer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_cinematographeres = pd.DataFrame({
    'Cinematographer': cinematographeres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_cinematographeres = df_cinematographeres[df_cinematographeres["Movies"] > 1]
df_cinematographeres = df_cinematographeres.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_cinematographeres[:(12 if len(df_cinematographeres)>12 else len(df_cinematographeres))], x="Movies", y="Cinematographer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Cinematographers '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/6_Most_Watched_Cinematographers.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Cinematographers
df = pd.read_json('/content/resultado.json')

top = df['cinematographer'].explode().value_counts()
unique_cinematographers = top[top > 1].index.tolist()

cinematographeres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for cinematographer in unique_cinematographers:
    search_results = df[df['cinematographer'].apply(lambda x: cinematographer in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    cinematographeres_lista.append(cinematographer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_cinematographeres = pd.DataFrame({
    'Cinematographer': cinematographeres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_cinematographeres = df_cinematographeres[df_cinematographeres["Movies"] > 1]
df_cinematographeres = df_cinematographeres.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_cinematographeres[:(12 if len(df_cinematographeres)>12 else len(df_cinematographeres))], x="Personal Rating", y="Cinematographer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Cinematographers '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/7_Favorite_Cinematographers.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Composers
df = pd.read_json('/content/resultado.json')

top = df['composer'].explode().value_counts()
unique_composers = top[top > 1].index.tolist()

composeres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for composer in unique_composers:
    search_results = df[df['composer'].apply(lambda x: composer in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    composeres_lista.append(composer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_composeres = pd.DataFrame({
    'Composer': composeres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_composeres = df_composeres[df_composeres["Movies"] > 1]
df_composeres = df_composeres.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_composeres[:(12 if len(df_composeres)>12 else len(df_composeres))], x="Movies", y="Composer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Composers '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/8_Most_Watched_Composer.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Composers
df = pd.read_json('/content/resultado.json')

top = df['composer'].explode().value_counts()
unique_composers = top[top > 1].index.tolist()

composeres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for composer in unique_composers:
    search_results = df[df['composer'].apply(lambda x: composer in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    composeres_lista.append(composer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_composeres = pd.DataFrame({
    'Composer': composeres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_composeres = df_composeres[df_composeres["Movies"] > 1]
df_composeres = df_composeres.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_composeres[:(12 if len(df_composeres)>12 else len(df_composeres))], x="Personal Rating", y="Composer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Composers '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/9_Favorite_Composer.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Writers
df = pd.read_json('/content/resultado.json')

top = df['writer'].explode().value_counts()
unique_writers = top[top > 1].index.tolist()

writeres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for writer in unique_writers:
    search_results = df[df['writer'].apply(lambda x: writer in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    writeres_lista.append(writer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_writeres = pd.DataFrame({
    'Writer': writeres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_writeres = df_writeres[df_writeres["Movies"] > 1]
df_writeres = df_writeres.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_writeres[:(12 if len(df_writeres)>12 else len(df_writeres))], x="Movies", y="Writer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Writers '+r'$(n \geq 2)$',  fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/10_Most_Watched_Writer.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Writers
df = pd.read_json('/content/resultado.json')

top = df['writer'].explode().value_counts()
unique_writers = top[top > 1].index.tolist()

writeres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for writer in unique_writers:
    search_results = df[df['writer'].apply(lambda x: writer in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    writeres_lista.append(writer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_writeres = pd.DataFrame({
    'Writer': writeres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_writeres = df_writeres[df_writeres["Movies"] > 1]
df_writeres = df_writeres.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_writeres[:(12 if len(df_writeres)>12 else len(df_writeres))], x="Personal Rating", y="Writer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Writers '+r'$(n \geq 2)$',  fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/11_Favorite_Writer.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Editors
df = pd.read_json('/content/resultado.json')

top = df['editor'].explode().value_counts()
unique_editors = top[top > 1].index.tolist()

editores_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for editor in unique_editors:
    search_results = df[df['editor'].apply(lambda x: editor in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    editores_lista.append(editor)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_editores = pd.DataFrame({
    'Editor': editores_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_editores = df_editores[df_editores["Movies"] > 1]
df_editores = df_editores.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_editores[:(12 if len(df_editores)>12 else len(df_editores))], x="Movies", y="Editor")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Editors '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.savefig("/content/img/12_Most_Watched_Editor.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Editors
df = pd.read_json('/content/resultado.json')

top = df['editor'].explode().value_counts()
unique_editors = top[top > 1].index.tolist()

editores_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for editor in unique_editors:
    search_results = df[df['editor'].apply(lambda x: editor in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    editores_lista.append(editor)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_editores = pd.DataFrame({
    'Editor': editores_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_editores = df_editores[df_editores["Movies"] > 1]
df_editores = df_editores.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_editores[:(12 if len(df_editores)>12 else len(df_editores))], x="Personal Rating", y="Editor")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Editors '+r'$(n \geq 2)$',  fontsize=11, y=1.01, loc="center")
plt.ylabel("")

plt.savefig("/content/img/13_Favorite_Editor.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Most Watched Producer	s
df = pd.read_json('/content/resultado.json')

top = df['producer'].explode().value_counts()
unique_producers = top[top > 1].index.tolist()

produceres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for producer	 in unique_producers:
    search_results = df[df['producer'].apply(lambda x: producer	 in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    produceres_lista.append(producer	)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_produceres = pd.DataFrame({
    'Producer	': produceres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_produceres = df_produceres[df_produceres["Movies"] > 1]
df_produceres = df_produceres.sort_values(by="Movies", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])
fig = sns.barplot(data=df_produceres[:(12 if len(df_produceres)>12 else len(df_produceres))], x="Movies", y="Producer	")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Most Watched Producers '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")
plt.xticks(np.arange(0, max(df_produceres["Movies"]) + 1, 1))

plt.savefig("/content/img/14_Most_Watched_Producers.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Favorite Producers
df = pd.read_json('/content/resultado.json')

top = df['producer'].explode().value_counts()
unique_producers = top[top > 1].index.tolist()

produceres_lista = []
movies_lista = []
average_rating_lista = []
personal_rating_lista = []

for producer	 in unique_producers:
    search_results = df[df['producer'].apply(lambda x: producer	 in x)]
    nota_personal = search_results['my_rating'].mean()
    nota = search_results['rating'].mean()
    nota = round(nota, 2) * 2
    nota_personal = round(nota_personal, 2) * 2
    n_movies = len(search_results)

    produceres_lista.append(producer)
    movies_lista.append(n_movies)
    average_rating_lista.append(nota)
    personal_rating_lista.append(nota_personal)

df_produceres = pd.DataFrame({
    'Producer': produceres_lista,
    'Movies': movies_lista,
    'Average Rating': average_rating_lista,
    'Personal Rating': personal_rating_lista
})


df_produceres = df_produceres[df_produceres["Movies"] > 1]
df_produceres = df_produceres.sort_values(by="Personal Rating", ascending=False).head(15)
sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[10,7])

fig = sns.barplot(data=df_produceres[:(12 if len(df_produceres)>12 else len(df_produceres))], x="Personal Rating", y="Producer")
fig.bar_label(fig.containers[0], padding=3, fontsize=11, color='white')
plt.title('Favorite Producers '+r'$(n \geq 2)$', fontsize=11, y=1.01, loc="center")
plt.ylabel("")

df_produceres
plt.savefig("/content/img/15_Favorite_Producers.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Top 12 | Most Watched Countries
f = open('/content/resultado.json')
df = pd.read_json(f)

df = df['country']
df = [item for sublist in df for item in sublist]
df = pd.DataFrame(df)
df.rename(columns={0:'Country'}, inplace=True)
df = df.value_counts().reset_index(name='Movie Count')

sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[13,7])
fig = sns.barplot(data=df[:12], x="Country", y="Movie Count")
fig.bar_label(fig.containers[0], fontsize=11, fontweight=500, color='white', label_type='edge')
plt.title("Top 12 | Most Watched Countries",fontsize=11)
plt.xlabel("")
plt.ylabel("")

# save plot figure
plt.savefig("/content/img/16_Top_12_Most_Watched_Countries.png", dpi=300, bbox_inches='tight')

In [ ]:
#@title Top 12  |  Most Watched Genre
f = open('/content/resultado.json')
df = pd.read_json(f)

df = df['genre']
df = [item for sublist in df for item in sublist]
df = pd.DataFrame(df)
df.rename(columns={0:'Genre'}, inplace=True)
df = df.value_counts().reset_index(name='Movie Count')

sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

plt.figure(figsize=[14,7])
fig = sns.barplot(data=df[:12], x="Genre", y="Movie Count")
fig.bar_label(fig.containers[0], fontsize=11, fontweight=500, color='white', label_type='edge')
plt.title("Top | 12 Most Watched Genres",fontsize=11)
plt.xlabel("")
plt.ylabel("")
# save plot figure
plt.savefig("/content/img/17_Top_12_Most_Watched_Genres.png", dpi=300, bbox_inches='tight')


In [ ]:
#@title Top 12 | Most Spoken Languages
f = open('/content/resultado.json')
df = pd.read_json(f)
df = df['language']
df = [item for sublist in df for item in sublist]
df = pd.DataFrame(df)
df.rename(columns={0:'Language'}, inplace=True)
df = df.value_counts().reset_index(name='Movie Count')


# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

sns.dark_palette("xkcd:golden",4)
plt.style.use("dark_background")
plt.figure(figsize=[17,6])

fig = sns.barplot(data=df[:12], x="Language", y="Movie Count")
fig.bar_label(fig.containers[0], fontsize=11, padding=2, color='white', label_type='edge')
plt.title("Top 12 | Most Spoken Languages",fontsize=11)
plt.xlabel("")
plt.ylabel("")
# save plot figure
plt.savefig("/content/img/18_Top_12_Most_Spoken_Languagues.png", dpi=300, bbox_inches='tight')


In [ ]:
#@title Frequent Collaborators (Director - Actor)
f = open('/content/resultado.json')
df = pd.read_json(f)

df = df.explode("director").explode("actor")

table = df[['director', 'actor']].copy()
table = table[table['director'] != table['actor']]
table = pd.DataFrame(table.value_counts().reset_index(name="Count"))
table = table[table['Count'] >= 2]
table['Director-Actor'] = table['director'] + ' - ' + table['actor']
table = table[['Director-Actor', 'Count']]
table = table[:10] if len(table) >= 10 else table
table.index.name = 'Rank'
table.index += 1

def render_table(data, col_width=3.0, row_height=0.5, font_size=11,
                     header_color='#fff', row_colors=['#000', '#222'], edge_color='w',
                     bbox=[0, 0, 2, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.to_records(), bbox=bbox, colLabels=data.columns.insert(0, data.index.name), **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='#000000')
            cell.set_facecolor('#ffffff')
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

def max_widths(data):
    measurer = np.vectorize(len)
    max_lengths = measurer(data.to_numpy().astype(str)).max(axis=0)
    max_lengths = np.insert(max_lengths, 0, len(str(len(df))))

    i = 0
    for name in data.columns.insert(0, df.index.name):
        if len(str(name)) > max_lengths[i]: max_lengths[i] = len(str(name))
        i += 1

    return max_lengths

widths = max_widths(table)

ax = render_table(table, header_columns=0, col_width=sum(widths)/25, colWidths=widths, cellLoc='center', header_color="b")
plt.title("Frequent Collaborators (Director - Actor)", fontsize=11, y=1.01, x=1, loc='center')
ax.get_figure().savefig('/content/img/19_Frequent_Collaborators_Director-Actors.png', dpi=300, bbox_inches='tight')


In [ ]:
#@title Top 10 | Watched Films with the Best Average Ratings
f = open('/content/resultado.json')
df = pd.read_json(f)

df = df[['title', 'rating','my_rating']]
df.columns = ['Title','Average Rating','Personal Rating']
df = df.sort_values('Average Rating', ascending=False).reset_index(drop=True)
df = df[:10].round(2)
df.reset_index(drop=True, inplace=True)
df['Average Rating']=df['Average Rating']*2
df['Personal Rating']=df['Personal Rating']*2
df.index.name = 'Rank'
df.index += 1
def render_table(data, col_width=3.0, row_height=0.525, font_size=11,
                     header_color='#fff', row_colors=['#000', '#222'], edge_color='w',
                     bbox=[0, 0, 2, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.to_records(), bbox=bbox, colLabels=df.columns.insert(0, df.index.name), **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='#000000')
            cell.set_facecolor('#ffffff')
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

def max_widths(data):
    measurer = np.vectorize(len)
    max_lengths = measurer(data.to_numpy().astype(str)).max(axis=0)
    max_lengths = np.insert(max_lengths, 0, len(str(len(df))))

    i = 0
    for name in df.columns.insert(0, df.index.name):
        if len(str(name)) > max_lengths[i]: max_lengths[i] = len(str(name))
        i += 1

    return max_lengths

widths = max_widths(df)

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

ax = render_table(df, header_columns=0, col_width=sum(widths)/25, colWidths=widths, cellLoc='center', header_color="teal")
plt.title("Top 10 | Watched Films with the Best Average Ratings", fontsize=11, y=1.02, x=1,loc='center')
ax.get_figure().savefig('/content/img/20_Top_10_Watched_Films_Best_Average_Ratings.png', dpi=300, bbox_inches='tight')


In [ ]:
#@title Top 10 | Watched Films with the Worst Average Ratings
f = open('/content/resultado.json')
df = pd.read_json(f)

df = df[['title', 'rating','my_rating']]
df.columns = ['Title','Average Rating','Personal Rating']
df = df.sort_values('Average Rating', ascending=True).reset_index(drop=True)
df = df[:10].round(2)
df.reset_index(drop=True, inplace=True)
df['Average Rating']=df['Average Rating']*2
df['Personal Rating']=df['Personal Rating']*2
df.index.name = 'Rank'
df.index += 1
def render_table(data, col_width=3.0, row_height=0.525, font_size=11,
                     header_color='#fff', row_colors=['#000', '#222'], edge_color='w',
                     bbox=[0, 0, 2, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')
    mpl_table = ax.table(cellText=data.to_records(), bbox=bbox, colLabels=df.columns.insert(0, df.index.name), **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='#000000')
            cell.set_facecolor('#ffffff')
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

def max_widths(data):
    measurer = np.vectorize(len)
    max_lengths = measurer(data.to_numpy().astype(str)).max(axis=0)
    max_lengths = np.insert(max_lengths, 0, len(str(len(df))))

    i = 0
    for name in df.columns.insert(0, df.index.name):
        if len(str(name)) > max_lengths[i]: max_lengths[i] = len(str(name))
        i += 1

    return max_lengths

widths = max_widths(df)

# configure font
rcParams['font.family'] = []
rcParams['font.sans-serif'] = []

ax = render_table(df, header_columns=0, col_width=sum(widths)/25, colWidths=widths, cellLoc='center', header_color="teal")
plt.title("Top 10 | Watched Films with the Worst Average Ratings", fontsize=12, y=1.02, x=1, loc='center')

ax.get_figure().savefig('/content/img/21_Top_10_Watched_Films_Worst_Average_Ratings.png', dpi=300, bbox_inches='tight')


In [ ]:
#@title Monthly Activity on Letterboxd

# Load your data
df = pd.read_json('/content/resultado.json')

# Get year-month information
dates = pd.to_datetime(df['watch_date'])
dates = dates.dt.to_period('M')

# Count occurrences
dates_count = dates.value_counts().sort_index()

# Create a list of month names
month_names = dates_count.index.strftime('%B')

sns.dark_palette("xkcd:golden", 4)
plt.style.use("dark_background")
plt.rcParams['font.family'] = []
plt.rcParams['font.serif'] = []

# Create the bar plot
plt.figure(figsize=(14, 7))
fig = sns.barplot(data=dates_count.reset_index(), x='index', y='watch_date')
fig.bar_label(fig.containers[0], fontsize=12, fontweight=500, color='white', label_type='edge')

# Set the title and axis labels
plt.title("Monthly Activity on Letterboxd", fontsize=11)
plt.xlabel("")
plt.ylabel("")

# Set the x-axis tick labels to the month names
plt.xticks(range(len(month_names)), month_names, )

plt.savefig("/content/img/22_Monthly_Activity.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
#@title Movies Watched Per Lustrum
# Load your data
df = pd.read_json('/content/resultado.json')

# Extract the decade from the 'released_year' column
df['decade'] = (df['released_year'] // 5) *5

# Group the data by decade and count the number of movies watched in each decade
decade_counts = df['decade'].value_counts().sort_index()

# Create a bar plot
plt.figure(figsize=(14, 7))
fig = sns.barplot(x=decade_counts.index, y=decade_counts.values)
fig.bar_label(fig.containers[0], fontsize=11, fontweight=500, color='white', label_type='edge')


sns.dark_palette("xkcd:golden", 4)
plt.style.use("dark_background")
plt.rcParams['font.family'] = []
plt.rcParams['font.serif'] = []

plt.title("Movies Watched Per Lustrum")


# Display the plot
plt.savefig("/content/img/23_Movies_Watched_Per_Lustrum.png", dpi=300, bbox_inches='tight')
plt.show()
# Calculate and display the mean rating per decade
mean_rating_per_decade = 2 *(df.groupby('decade')['rating'].mean().sort_index())

# Display the table
mean_rating_table = pd.DataFrame({'Decade': mean_rating_per_decade.index, 'Mean Rating': mean_rating_per_decade.values})
display(mean_rating_table)

In [ ]:
#@title Distribution of Personal Rating
# Supongamos que tienes un DataFrame 'df' con una columna llamada 'my_rating'
f = open('/content/resultado.json')
df = pd.read_json(f)

# Calcular la distribución de las calificaciones personales
value_counts = df['my_rating'].value_counts().sort_index()

# Crear un gráfico de barras
plt.figure(figsize=(14, 7))
fig = sns.barplot(x=value_counts.index, y=value_counts.values)

# Agregar el símbolo de estrella al final de los valores en el eje x
x_labels = [f'{rating} \u2605' for rating in value_counts.index]

# Establecer las etiquetas personalizadas en el eje x
plt.xticks(range(len(value_counts.index)), labels=x_labels, rotation=0, fontsize=11)

fig.bar_label(fig.containers[0], fontsize=11, color='white', label_type='edge')

sns.dark_palette("xkcd:golden", 4)
plt.style.use("dark_background")
plt.rcParams['font.family'] = []
plt.rcParams['font.serif'] = []
plt.title('Distribution of Personal Rating (Over 5)')
plt.ylabel('Movies')

# Mostrar el gráfico
plt.savefig("/content/img/24_Distribution_of_Personal_Rating.png", dpi=300, bbox_inches='tight')
plt.show()






In [ ]:
#@title Save Data in Zip
# Ruta de la carpeta que deseas comprimir
carpeta = '/content/img'

# Nombre del archivo ZIP resultante
archivo_zip = '/content/letterboxd_stats.zip'

# Crear un archivo ZIP
with zipfile.ZipFile(archivo_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Recorrer todos los archivos y subcarpetas en la carpeta
    for raiz, directorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            # Ruta completa del archivo
            ruta_completa = os.path.join(raiz, archivo)
            # Ruta relativa del archivo dentro de la carpeta
            ruta_relativa = os.path.relpath(ruta_completa, carpeta)
            # Agregar el archivo al archivo ZIP con su ruta relativa
            zipf.write(ruta_completa, ruta_relativa)

print(f"Archivo ZIP '{archivo_zip}' generado exitosamente.")
files.download(archivo_zip)
